# Importing Useful Libraries

-  Simbench installation through pip: <b> pip install simbench </b>(for more information or non pip installation https://simbench.readthedocs.io/en/stable/about/installation.html)
-  Pandapower installation through pip: <b> pip install pandapower </b> (for more information https://www.pandapower.org/)  
-  Sklearn installation through pip: <b> pip install -U scikit-learn </b> (for more information https://scikit-learn.org/stable/install.html)
-  Matplotlib installation through pip: <b> python -m pip install -U matplotlib </b> (for more information https://matplotlib.org/stable/install/index.html)          
-  Pandas installation through pip: <b> pip install pandas </b>
-  Numpy installation through pip: <b> pip install numpy </b>                                          

In [ ]:
import simbench as sb
import pandapower.timeseries as ts
from pandapower.timeseries.data_sources.frame_data import DFData
from pandapower.control.controller.const_control import ConstControl

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
# from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as clrs

# Dataset loading: Importing Network to be Used

Simbench allows us to obtain networks with profiles for 1 year of 15 minutes timesteps

In [ ]:
grid_code = '1-HV-urban--0-no_sw'
net = sb.get_simbench_net(grid_code)     # Get network
print(net)                               # Print Netowrk to see Components
profiles = sb.get_absolute_values(net, profiles_instead_of_study_cases=True)  # Get full profiles with 1 year information on the network components

# Please Ignore the Returned Warnings

Get load active and reactive powers, generator active powers and per unit voltages.Those are the sets of features of the input data. Each column of those sets represents a different load/static generator.

In [ ]:
# Obtain the Profiles
load_p = profiles[('load','p_mw')]
load_q = profiles[('load','q_mvar')]

sgen_p = profiles[('sgen','p_mw')]
print(f"First 5 timesteps have loads with active powers in MW of: {load_p.head()}")
# For Computational Time Reduction, only use data from the first 2 months (leap year)
load_p = load_p.iloc[:4*24*60, :]
load_q = load_q.iloc[:4*24*60, :]
sgen_p = sgen_p.iloc[:4*24*60, :]

# Task 1: Generic Information Extraction

1) Find the maximum active power consumption per load within the load profile (per column) <br>
2) Find the maximum active power generation from the static generators profile <br>
3) Find the maximum apparent power consumption magnitude from the load no.0 active and reactive power consumption profiles (Hint: Load no.0 is the 1st Column of the load_p and load_q dataframes). In your result, also include the timestep index of the found maximum apparent power.  <br>

In [ ]:
# Answer

#1)
max_p__per_load =
print(f"The maximum active power consumed by each load {max_p__per_load}")
#2)
max_p_sgen =
print(f"The maximum active power generation is {max_p_sgen}")

#3)

max_s_timestep =
max_s =
print(f"The maximum aparent power of the load 0 is {max_s} and was found in timestep {max_s_timestep}")

# Running the Timeseries Calculation to Create All Labels for the Datapoints Given In the Profiles

The following section runs a timeseries simulation based on the active and reactive load powers, and active static generator power scenarios shown previously. The results from the powerflows solved for each timestep are stored under the Chapter6 folder.

In [ ]:
ds = DFData(sgen_p)
ConstControl(net, "sgen", 'p_mw', element_index=net.sgen.index, profile_name=sgen_p.columns, data_source=ds)
ds = DFData(load_p)
ConstControl(net, "load", 'p_mw', element_index=net.load.index, profile_name=load_p.columns, data_source=ds)
ds = DFData(load_q)
ConstControl(net, "load", 'q_mvar', element_index=net.load.index, profile_name=load_q.columns, data_source=ds)

ts.OutputWriter(net, output_path='./Chapter6/', output_file_type=".csv")
ts.run_time_series.run_timeseries(net, continue_on_divergence=True )

# Loading  of  simulation  results  and  generation  of  machine  learning  model datasets

Geting the simulated loading percentage as the output to be used in the machine learning model

In [ ]:
line_loading_percentages = pd.read_csv("./Chapter6/res_line/loading_percent.csv",sep=';')
X_full = pd.concat([sgen_p, load_p, load_q], axis=1)

print(X_full.head())
X_max = X_full.max()
print(f"Maximum value in X_full is {X_max.max()}")
print(f"Maximum index in X_full is {X_max.idxmax()}")

print(f"Average value in X_full is {X_full.mean()}")
print(f"STD value in X_full is {X_full.std()}")



Typically, within power grid there is no full observability of the consumption and generation at the TSO levels.
Therefore, to incorporate this we will randomly remove some of the static geneator and load data.

In [ ]:
observable_generator_percentage = 10.
observable_load_percentage = 10.

rng = np.random.RandomState(3065)
range_gen_list = np.arange(0, len(sgen_p.columns))
range_load_list = np.arange(0, len(load_p.columns))

rng.shuffle(range_gen_list)
rng.shuffle(range_load_list)

known_gen_list = range_gen_list[:int(observable_generator_percentage*len(sgen_p.columns)/100)]
known_load_list = range_load_list[:int(observable_load_percentage*len(load_p.columns)/100)]

X = pd.concat([sgen_p.iloc[:, known_gen_list], load_p.iloc[:, known_load_list], load_q.iloc[:, known_load_list]], axis=1)
print(X.shape, X_full.shape)

# Task 2: Regression Task Target Set Creation

1) Find which is the highest line loading percentage within the dataset <br>
2) Find which line ("line_Y") has the highest loading percentage within the dataset <br>
3) Save all loading percentage values of line "line_Y" as the vector 'y'. The chosen line will be used as the dataset targets to be predicted by the regression problems. <br>

In [ ]:
#Answers
#1),2)
line_Y =
max_value =
print(f"Highest load percentage is {max_value} and is observed on line {line_Y}")

#3)
y =
print(f"The regression target set is: {y}")

# Task 3: Matplotlib introduction

1) Plot the data labels in a line plot with x-axis as the minutes past compared to the first datapoint (remember: the dataset was created in 15 minute intervals). Add axis labels and appropriate title to the figure. The shape of the figure should have 15 inch width and 5 inch height.

In [ ]:
#Answer
#1)
x_axis =
y_axis =
plt.figure(figsize=(15,5))
plt.plot(x_axis, y_axis)
plt.xlabel("timestep [m]")
plt.ylabel("loading percentage [%]")
plt.title("Label set")
plt.show()

# Creation of a regression plotting function for later results

In [ ]:
def plot_regression_prediciton(y_pred, y_real, title):
    plt.figure(figsize=(20,5))
    plt.plot(y_real.index, y_pred, 'o')
    plt.plot(y_real, 'o')
    plt.xlabel('datapoints')
    plt.ylabel('line loading')
    plt.title(title)
    plt.legend(['Prediction', 'Actual'])
    plt.show()

# Data Preprocessing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3065, shuffle=True)

Standardize generally means changing the values so that the distribution’s standard deviation equals one. Scaling is often implied.
StandardScaler standardizes a feature by subtracting the mean and then scaling to unit variance. Unit variance means dividing all the values by the standard deviation. StandardScaler results in a distribution with a standard deviation equal to 1. The variance is equal to 1 also, because variance = standard deviation squared. And 1 squared = 1.
StandardScaler makes the mean of the distribution 0. About 68% of the values will lie be between -1 and 1.

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Regression: 1.Linear Regression

LinearRegression fits a linear model with coefficients w = (w1, …, wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.
More information in: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html


In [ ]:
linear_reg = LinearRegression().fit(X_train, y_train)
#Score is R^2
linear_reg.score(X_train, y_train)

In [ ]:
y_pred_lr = linear_reg.predict(X_test)
#Score is R^2
linear_reg.score(X_test, y_test)

In [ ]:
plot_regression_prediciton(linear_reg.predict(X_train), y_train, "Linear Regression Training Results")

In [ ]:
plot_regression_prediciton(y_pred_lr, y_test, "Linear Regression Test Results")

# Regression: 2. Support Vectors Regression (SVR)

More information at: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR

In [ ]:
##Hyperparameters

#1. Specifies the kernel type to be used in the algorithm. It must be one of ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ or a
#callable. If none is given, ‘rbf’ will be used. If a callable is given it is used to precompute the kernel matrix.
kernel = 'rbf' #DO NOT CHANGE

#2. Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.
degree = 3 #DO NOT CHANGE

#3. Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
gamma = 'scale' # {‘scale’, ‘auto’} or float DO NOT CHANGE

#4. Independent term in kernel function. It is only significant in ‘poly’ and ‘sigmoid’.
coef0 = 0.0 #float DO NOT CHANGE

#5. Tolerance for stopping criterion.
tol = 1e-3 # DO NOT CHANGE

#6. Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive.
#The penalty is a squared l2 penalty.
C = 1.

#7. Epsilon in the epsilon-SVR model. It specifies the epsilon-tube within which no penalty is associated in the training
#loss function with points predicted within a distance epsilon from the actual value.
epsilon = 0.1 # float DO NOT CHANGE

#8. Whether to use the shrinking heuristic
shrinking = True # bool DO NOT CHANGE

#9. Hard limit on iterations within solver, or -1 for no limit.
max_iter = 1000 # DO NOT CHANGE

As you can see, there are many hyperparameters which can be tuned to affect the performance of the model. In this task, you are asked only to fine tune the hyperparameter  <b>C</b> between values of </b>0.1 and 5</b>. Change the parameter value in 3 different models and document the performance differences between them.  

In [ ]:
sv_reg = svm.SVR(kernel=kernel, C=C, epsilon=epsilon , gamma=gamma ,max_iter= max_iter).fit(X_train, y_train)
sv_reg.score(X_train, y_train)

In [ ]:
y_pred_svr = sv_reg.predict(X_test)
sv_reg.score(X_test, y_test)

In [ ]:
plot_regression_prediciton(sv_reg.predict(X_train), y_train, "Support Vector Regression Training Results")

In [ ]:
plot_regression_prediciton(y_pred_svr, y_test, "Support Vector Regression Test Results")

# Task 4: SVR Hyperparameter Tuning

1) Change the hyperparameter 'C' between values of 0.1 and 5 in order to improve the training error. Report the resulted training and test errors for 3 different combinations. Based on your results, which model would you choose and why?

In [ ]:
#Answers

print(f"A. For the hyperparameter C = ___ , the resulted training and test errors are {} and {}")

print(f"B. For the hyperparameter C = ___ , the resulted training and test errors are {} and {}")

print(f"C. For the hyperparameter C = ___ , the resulted training and test errors are {} and {}")

print(f"My chosen hyperparameter C would be ___________________, because _____________")


# Regression: 3. Decision Tree Regression

In [ ]:
##Hyperparameters

#.0 Random State is used to repeat same results whenever the model is run
random_state = 3065 #DO NOT CHANGE

#1. The function to measure the quality of a split. Supported criteria are “squared_error” for the mean squared error,
#which is equal to variance reduction as feature selection criterion and minimizes the L2 loss using the mean of each terminal
#node, “friedman_mse”, which uses mean squared error with Friedman’s improvement score for potential splits,
#“absolute_error” for the mean absolute error, which minimizes the L1 loss using the median of each terminal node,
#and “poisson” which uses reduction in Poisson deviance to find splits.
criterion = "squared_error" #{“squared_error”, “friedman_mse”, “absolute_error”, “poisson”}

#2. The strategy used to choose the split at each node. Supported strategies are “best” to choose the best split and “random” to choose the best random split.
splitter = 'best' #{“best”, “random”}

#3. The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
max_depth = None # int

#4. The minimum number of samples required to split an internal node:
min_samples_split = 3 # int or float

#5. The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves
#at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.
min_samples_leaf = 1 # int or float

#6. The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node.
#Samples have equal weight when sample_weight is not provided.
min_weight_fraction_leaf = 0. # float

#7. The number of features to consider when looking for the best split:
max_features = 10 # int or float or {"auto, "sqrt", "log2"}, None => n_features

#8. Grow a tree with max_leaf_nodes in best-first fashion. Best nodes are defined as relative reduction in impurity. If None then unlimited number of leaf nodes.
max_leaf_nodes = None # int

#9. A node will be split if this split induces a decrease of the impurity greater than or equal to this value.
min_impurity_decrease = 0.0 # float

#10. Complexity parameter used for Minimal Cost-Complexity Pruning. The subtree with the largest cost complexity that is smaller than ccp_alpha will be chosen.
#By default, no pruning is performed.
ccp_alpha = 0.0 # non negative float


As you can see, there are many hyperparameters which can be tuned to affect the performance of the model. In this task, you are asked only to fine tune the hyperparameter  <b>min_samples_split</b> between values of </b>2 and 50</b>. Change the parameter value in 3 different models and document the performance differences between them.  

In [ ]:
DT_reg = DecisionTreeRegressor(random_state=random_state, splitter=splitter, max_depth=max_depth, min_samples_split=min_samples_split,
                               min_samples_leaf=min_samples_leaf, min_weight_fraction_leaf=min_weight_fraction_leaf, max_features=max_features,
                               max_leaf_nodes=max_leaf_nodes, min_impurity_decrease=min_impurity_decrease, ccp_alpha=ccp_alpha)
DT_reg.fit(X_train,y_train)
DT_reg.score(X_train, y_train)

In [ ]:
y_pred_dtr = DT_reg.predict(X_test)
DT_reg.score(X_test, y_test)

In [ ]:
plot_regression_prediciton(DT_reg.predict(X_train), y_train, "Decision Tree Regression Training Results")

In [ ]:
plot_regression_prediciton(y_pred_dtr, y_test, "Decision Tree Regression Test Results")

# Task 5: Decision Tree Regressor Hyperparameter Tuning

1) Change the hyperparameter 'min_samples_split' in order to improve the current model (if you are not able to improve the model explain why). Report the resulted training and test errors for 3 different combinations between 2 and 50. Based on your results, which model would you choose and why?

In [ ]:
#Answers

print(f"A. For the hyperparameter 'min_samples_split'= ____, the resulted training and test errors are {} and {}")

print(f"B. For the hyperparameters 'min_samples_split'= ____, the resulted training and test errors are {} and {}")

print(f"C. For the hyperparameters 'min_samples_split'= ____, the resulted training and test errors are {} and {}")

print(f"My chosen hyperparameter 'min_samples_split' would be ___________________, because _____________")

# Classification Label Set Creation

The line in the context of these lecture will be considered as within dangerous limits when it's
loading percentage is equal or higher than 60%. Hence, the target label of each datapoint needs to be determined. Loading percentages higher than 60% will be labeled as 1, while the rest of the line loading percentages will be labeled as 0.

# Task 6: Creation of Classification Label Set

1) Using the y_train set, create the classification labels vector y_train_class with values 0 and 1 as explained above. Print the ratio of training set safe datapoints with respect to all the training set data points <br>
2) Using the y_test set, create the classification labels vector y_test_class with values 0 and 1 as explained above. Print the ratio of test set safe datapoints with respect to all the test set data point <br>


In [ ]:
#Answers
limit =60.


y_train_class =
y_test_class =
safe_train_ratio = 100. * float(sum(y_train_class))/len(y_train_class)
safe_test_ratio = 100. * float(sum(y_test_class))/len(y_test_class)

print(f"Out of {len(y_train_class)} Training data, the {safe_train_ratio} % are representing 'safe' line percentage")
print(f"Out of {len(y_test_class)} Test data, the {safe_test_ratio} % are representing 'safe' line percentage")

# Classification Printing Function Creation

In [ ]:
def plot_classification_prediction(y_real, y_pred_class, y_real_class, title):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 5))
    axes[0].scatter(y_real.index, y_real, c=y_pred_class, cmap=clrs.ListedColormap(['blue', 'red']))
    axes[1].scatter(y_real.index, y_real, c=y_real_class, cmap=clrs.ListedColormap(['blue', 'red']))
    axes[0].set_xlabel('datapoints')
    axes[0].set_ylabel('line loading')
    axes[1].set_xlabel('datapoints')
    axes[1].set_ylabel('line loading')
    axes[0].title.set_text('Predicted Classes')
    axes[1].title.set_text('Actual Classes')
    fig.suptitle(title, fontsize=16)
    plt.show()

# Classification 1: Linear Classification - Logistic Regression Classifier

This class implements regularized logistic regression using the ‘liblinear’ library, ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ solvers. Note that regularization is applied by default. It can handle both dense and sparse input. Use C-ordered arrays or CSR matrices containing 64-bit floats for optimal performance; any other input format will be converted (and copied).
More information at: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

In [ ]:
# Hyperparameters

#.0 Random State is used to repeat same results whenever the model is run
random_state = 3065 #DO NOT CHANGE

#1. Specify the norm of the penalty:
penalty = 'l2' #'l1', 'l2', ‘elasticnet’, ‘none’

#2. Dual or primal formulation. Dual formulation is only implemented for l2 penalty with liblinear solver.
#Prefer dual=False when n_samples > n_features.
dual = False # Do not change in our case given the above inequality

#3. Tolerance for stopping criterion.
tol = 1e-3 #float

#4. Inverse of regularization strength; must be a positive float.
#Like in support vector machines, smaller values specify stronger regularization.
C = 1.  #float

#5. Specifies if a constant (a.k.a. bias or intercept) should be added to the decision function.
fit_intercept = True #bool

#6. Useful only when the solver ‘liblinear’ is used and self.fit_intercept is set to True. In this case,
#x becomes [x, self.intercept_scaling], i.e. a “synthetic” feature with constant value equal to intercept_scaling
#is appended to the instance vector. The intercept becomes intercept_scaling * synthetic_feature_weight.
intercept_scaling = 1. #float

#7. Weights associated with classes in the form {class_label: weight}. If not given, all classes are supposed to have weight one.
#The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies
#in the input data as n_samples / (n_classes * np.bincount(y)).
class_weight = None # dict or 'balanced'


In [ ]:
lr_class = LogisticRegression(random_state=random_state, penalty=penalty, dual=dual, tol=tol,
                              C=C, fit_intercept=fit_intercept, intercept_scaling=intercept_scaling,
                              class_weight=class_weight)
lr_class.fit(X_train, y_train_class)

#Return the mean accuracy on the given test data and labels
lr_class.score(X_train, y_train_class)

In [ ]:
plot_classification_prediction(y_train, lr_class.predict(X_train), y_train_class, "Linear Regression Training Set Classification")

The left figure plotted above illustrates the datapoints which were predicted as safe in blue, and the ones which were predicted as dangerous in red. The right figure illustrates the data point which were actually safe in blue, and the ones which were actually dangerous in red. Comparing those two graphs it is obvious that the majority of the datapoints is correctly classified but there are still some missclassification instances.

To help us visualize the amount of mistclassified data points, but also to have more insights on the performance of the model with respect to each class, the confusion matrix is created below. This matrix, shows information such as how many of the actually safe (class 0 ) datapoints were identified by the model, and how many were not. Similarly, it also shows how many of the actually dangerous datapoints were identified and how many were not. Those values can be used to obtain metrics such as Precision, Recall, F1 score and Accuracy.

In [ ]:
ConfusionMatrixDisplay.from_estimator(lr_class, X_train, y_train_class)

Although the previous graphs can showcase how the model performs on the training data, that performance evaluation might not be a good representation on how the model would perform to new, unseen data. Therefore, we also create the same figures for the test data, which represent a more accurate model performance to new data.

In [ ]:
y_pred_lrc = lr_class.predict(X_test)
lr_class.score(X_test, y_test_class)

In [ ]:
plot_classification_prediction(y_test, y_pred_lrc, y_test_class, "Linear Regression Test Set Classification")

In [ ]:
ConfusionMatrixDisplay.from_estimator(lr_class, X_test, y_test_class)

# Task 7: Creation of SVM Classification Model

1) Create an SVM classifier using scikit learn (Hint: Copy Implementation of Classifier 1 and then visit https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC to change model function, and hyperparameters) <br>
2) Train the SVM classifier (Hint: Similar to Classifier 1) <br>
3) Report training & test scores <br>
4) Plot the classification results for the test set using the plot_classification_prediction() function <br>
5) Plot the confusion matrix for the test set <br>

In [ ]:
# Answer

# (Optional) Task 8: Creation of Decision Tree Classification Model


1) Develop a Decision Tree classifier using scikit learn

2) Train the Decision Tree classifier (Hint: Similar to Classifier 1)

3) Report training & test scores

4) Plot the classification results for the test set using the plot classification prediction() function

5) Plot the confusion matrix for the test set